In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("miso_tranche2_limits.csv", index_col=0)

In [3]:
df = df.assign(r_rr = ['-999']*len(df))

In [4]:
df = df.drop(columns=['r','rr','geometry_r','geometry_rr','voltage','ll']).rename(columns={'ba_r':'r',
                                                                                      'ba_rr':'rr'})

In [5]:
df = df.query("r != rr")

In [6]:
for index, row in df.iterrows():
    df.at[index,'r_rr'] = f"{row.r}-{row.rr}"

In [7]:
df_grouped = df.set_index(['r','rr']).groupby('r_rr').sum()


In [8]:
df_grouped['r'] = [int(r[0]) for r in df_grouped.index.str.split('-').values]
df_grouped['rr'] = [int(r[1]) for r in df_grouped.index.str.split('-').values]

In [9]:
def fix_date(x):
    if x > 4000:
        return int(np.ceil(x/2))
    else:
        return int(x)

In [10]:
df_grouped['isd'] = df_grouped['isd'].apply(fix_date)

In [11]:
baseline_file = "transmission_capacity_future_ba_baseline.csv"
baseline_df = pd.read_csv(baseline_file).fillna(-999)

In [12]:
baseline_df[['t','MW']] = baseline_df[['t','MW']].astype(int)

In [13]:
def prepend_p(x):
    return 'p' + str(x)

In [14]:
df_grouped['r'] = df_grouped['r'].apply(prepend_p)
df_grouped['rr'] = df_grouped['rr'].apply(prepend_p)

In [15]:
df_grouped= df_grouped.assign(status=['Certain']*len(df_grouped),
                  trtype=['AC']*len(df_grouped),
                  notes=['MISO Tranche 2.1']*len(df_grouped))\
          .reset_index(drop=True)\
          .rename(columns={'isd':'t'})

In [16]:
df_grouped[['t','MW']] = df_grouped[['t','MW']].astype(int)

In [17]:
baseline_df

,r,rr,status,trtype,t,MW,notes
0,p61,p62,Certain,AC,2010,1021,-999
1,p61,p63,Certain,AC,2010,387,-999
2,p12,p13,Certain,AC,2011,1262,-999
3,p12,p15,Certain,AC,2011,765,-999
4,p60,p63,Certain,AC,2011,606,-999
...,...,...,...,...,...,...,...
58,# p24,p25,Certain,LCC,2030,3000,TransWest Express
59,# p5,p15,Certain,AC,2030,1732,Boardman to Hemingway
60,### ^^^,-999,-999,-999,-999,-999,-999
61,p80,p104,Possible,AC,0,100000,Allows Chicago to connect to rest of PJM


In [18]:
updated_baseline = pd.concat([baseline_df, df_grouped])

In [19]:
updated_baseline = updated_baseline.fillna(-999).replace(-999, "")

In [20]:
updated_baseline.to_csv(baseline_file, index=False)